In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os

In [22]:
from bikesharing.ml_logic.data import get_data_with_cache
from bikesharing.params import *

- Remove duplicates
- Deal with missing values
- Scale the features
- Encode features
- Perform cyclical engineering

In [27]:
dfs = []
meta = {'years':[], 'n_columns':[], 'n_rows':[]}
for year in range(2019,2023,1):
    df = pd.read_csv(f'../raw_data/MVG_Rad_Fahrten_{year}.csv', sep=';')
    cols = [col.strip() for col in df.columns]
    df.columns = cols
    dfs.append(df)
    meta['years'].append(year)
    meta['n_columns'].append(df.shape[1])
    meta['n_rows'].append(df.shape[0])

/tmp/ipykernel_1865/1700516227.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'../raw_data/MVG_Rad_Fahrten_{year}.csv', sep=';')
/tmp/ipykernel_1865/1700516227.py:4: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'../raw_data/MVG_Rad_Fahrten_{year}.csv', sep=';')
/tmp/ipykernel_1865/1700516227.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'../raw_data/MVG_Rad_Fahrten_{year}.csv', sep=';')


In [4]:
len(dfs)

4

In [4]:
for df in dfs:
    print(df.shape)

(753678, 11)
(721752, 11)
(619573, 11)
(709144, 11)


In [28]:
data = pd.concat(dfs, axis=0)

In [29]:
def pre_process_df(df):
  #vstrip column names
  cols = [col.strip() for col in df.columns]
  df.columns = cols

  # remove column 'Row'
  if 'Row' in df.columns:
    df.drop(columns='Row', inplace=True)

  # select relevant columns only
  #df = df[['STARTTIME', 'STARTLAT', 'STARTLON', 'RENTAL_IS_STATION']].copy()
  
  # make string replacements values
  df_obj = df.select_dtypes(include='object')
  df[df_obj.columns] = df_obj.applymap(lambda x: x.strip().replace(',', '.') if isinstance(x, str) else x)

  # handle datetime
  df.STARTTIME = pd.to_datetime(df.STARTTIME)

  # handle numeric columns
  df.replace('NULL', np.NAN, inplace=True)
  df.replace('', np.NAN, inplace=True)
  df[['STARTLAT', 'STARTLON', 'RENTAL_IS_STATION','ENDLAT', 'ENDLON', 'RETURN_IS_STATION']] = df[['STARTLAT', 'STARTLON', 'RENTAL_IS_STATION','ENDLAT', 'ENDLON', 'RETURN_IS_STATION']].astype(np.float32)
  
  return df

In [30]:
data = pre_process_df(data)

In [15]:
def load_data_to_bq(
        data: pd.DataFrame,
        gcp_project:str,
        bq_dataset:str,
        table: str,
        truncate: bool
    ) -> None:
    """
    - Save the DataFrame to BigQuery
    - Empty the table beforehand if `truncate` is True, append otherwise
    """

    assert isinstance(data, pd.DataFrame)
    full_table_name = f"{gcp_project}.{bq_dataset}.{table}"

    #print(Fore.BLUE + f"\nSave data to BigQuery @ {full_table_name}...:" + Style.RESET_ALL)


    # reset column names
    data.columns = [f'_{col}' if isinstance(col, int) else col for col in data.columns]

    # Load data onto full_table_name
    client = bigquery.Client()

    write_mode = "WRITE_TRUNCATE" if truncate else "WRITE_APPEND"
    job_config = bigquery.LoadJobConfig(write_disposition=write_mode)

    job = client.load_table_from_dataframe(data, full_table_name, job_config=job_config)

    # 🎯 HINT for "*** TypeError: expected bytes, int found":
    # After preprocessing the data, your original column names are gone (print it to check),
    # so ensure that your column names are *strings* that start with either
    # a *letter* or an *underscore*, as BQ does not accept anything else

    print(f"✅ Data saved to bigquery, with shape {data.shape}")


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2804147 entries, 0 to 709143
Data columns (total 10 columns):
 #   Column               Dtype         
---  ------               -----         
 0   STARTTIME            datetime64[ns]
 1   ENDTIME              object        
 2   STARTLAT             float32       
 3   STARTLON             float32       
 4   ENDLAT               float32       
 5   ENDLON               float32       
 6   RENTAL_IS_STATION    float32       
 7   RENTAL_STATION_NAME  object        
 8   RETURN_IS_STATION    float32       
 9   RETURN_STATION_NAME  object        
dtypes: datetime64[ns](1), float32(6), object(3)
memory usage: 171.2+ MB


In [32]:
load_data_to_bq(
        data=data,
        gcp_project=os.environ.get("GCP_PROJECT"),
        bq_dataset=os.environ.get("BQ_DATASET"),
        table='raw_data_mvg',
        truncate=True
    )

✅ Data saved to bigquery, with shape (2804147, 10)


In [23]:
query =f'''
        SELECT *
        FROM `{GCP_PROJECT}.{BQ_DATASET}.raw_data_mvg`
    '''

In [15]:
df = get_data_with_cache(GCP_PROJECT, query=query, cache_dir=LOCAL_DATA_PATH)
df.shape

TypeError: expected str, bytes or os.PathLike object, not NoneType